In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex Pipelines: Vertex AI Custom Job training wrapper

## Overview
This notebook shows how to use the `run_as_vertex_ai_custom_job` wrapper to convert any component to run it as a Vertex AI custom job. This allows users to take advantage of the vertical and horizontal scaling for computation heavy tasks on Vertex AI. This requires that the underlying component has built in support distributed computation. To learn more about Vertex AI Custom Job see [Vertex AI Custom Training](https://cloud.google.com/vertex-ai/docs/training/custom-training). 


For `run_as_vertex_ai_custom_job` interface please see the [souce code here](https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/experimental/custom_job/custom_job.py#L30).

### Install additional packages

In [ ]:
!pip3 install  -U "git+https://github.com/kubeflow/pipelines.git#egg=kfp&subdirectory=sdk/python" -q
!pip3 install  -U google-cloud-pipeline-components -q
!pip3 install  -U google-cloud-aiplatform -q

## Before you begin
### Set your Project ID and Pipeline Root


In [ ]:
PROJECT_ID = "python-docs-samples-tests"
REGION = "us-central1"

### Import libraries

In [ ]:
import google_cloud_pipeline_components as gcpc
from kfp.v2 import components, compiler, dsl

## Create a component, convert it to a custom job to use in a pipeline.
Create a simple component that takes an input and produces an output

In [ ]:
producer_op = components.load_component_from_text(
    """
name: Producer
inputs:
- {name: input_text, type: String, description: 'Represents an input parameter.'}
outputs:
- {name: output_value, type: String, description: 'Represents an output paramter.'}
implementation:
  container:
    image: google/cloud-sdk:latest
    command:
    - sh
    - -c
    - |
      set -e -x
      echo "$0, this is an output parameter" | gsutil cp - "$1"
    - {inputValue: input_text}
    - {outputPath: output_value}
"""
)

Convert the component to a Vertex AI Custom Job

In [ ]:
# This is a temporary work around for v0.1.4 to point to the correct base image for custom_job
gcpc.experimental.custom_job.custom_job._DEFAULT_CUSTOM_JOB_CONTAINER_IMAGE = 'gcr.io/ml-pipeline/gcp-launcher:0.1.4'

custom_job_producer_op= gcpc.experimental.custom_job.custom_job.run_as_vertex_ai_custom_job(producer_op)

Define the pipeline:

In [ ]:
@dsl.pipeline(pipeline_root='', name='custom-job-sample-pipeline')
def pipeline(text: str = 'message'):
    custom_producer_task = custom_job_producer_op(input_text=text, project=PROJECT_ID ,location=REGION)

You can proceed to complie and run the pipeline from here as usual. 